In [1]:
import sys
sys.path.append("..")
import torch
from transformers import HfArgumentParser
import numpy as np

from lib.arguments import DataTrainingArguments, ModelArguments, TrainArguments
from lib.datasets.rvlcdip import RVLCDIPGraphClassification

from lib.datasets import selector as ds  
from lib.datasets import data_collator as dc
import lib.models.selector as ms
import lib.datasets.data_utils as du

from lib.datasets.tokenizer import GraphLayout_Tokenizer
from lib.models.modeling import MODEL

device = "cuda"

In [2]:
config = "../config/rvlcdip/rvlcdip.json"
mode = 'test'

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainArguments))
model_args, data_args, training_args = parser.parse_json_file(config)
tokenizer = GraphLayout_Tokenizer (data_args)
collator = dc.Collater(None, None)

test_dataset = RVLCDIPGraphClassification (data_args, mode=mode, tokenizer= tokenizer, start=0, end = 100, preprocess_flag= False)
loader = dc.DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn = collator, num_workers = 1)   

Number of test samples: 40000
Total processed files: 38534
The lenght of test is: 38534


In [5]:
pretrained_model_path = "../ckpt/last.ckpt"

model = MODEL (model_args, data_args, training_args,  use_metric = True)
model_dict = torch.load(pretrained_model_path)['state_dict']
model.load_state_dict(model_dict, strict=False)
model = model.model.eval()
model = model.to (device)

In [6]:
for batch in (loader):
    batch.to(device)
    output = model (batch)
    break

/home/mohbaf/anaconda3/envs/pyg/lib/python3.7/site-packages/transformers/modeling_utils.py:811: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning
